In [3]:
import pandas as pd

In [4]:
from collections import Counter
from nltk.corpus import stopwords
import string

In [5]:
from nltk.tokenize import word_tokenize, sent_tokenize

In [30]:
import re

In [6]:
tweet_df = pd.read_csv('./data/tweet_df_fac.csv')

In [7]:
dsa_plus = ['DSA','Alphabet Left']

In [8]:
add_stop = ['local', 'law', 'chair', 'land', 'comittee','application','oversight','committee','application','number','land use',
            'resolution','communication','new','york','city','council','councilmember','district','amend','code','administrative',
            'relation', 'calling', 'state', 'requiring', 'department', 'pass', 'sign', 'block', 'legislature', 'charter', 'governor',
            'section', 'property', 'lot', 'pursuant', 'approving', 'upon', 'establishing', 'borough', 'certain', 'commission', 'amendment', 
            'public', 'act', 'located', 'program', 'real', 'c', 'n','legislation', 'exemption', 'report', 'submitted','introduce',
            'preconsidered','manhattan','staten','island','bronx','brooklyn','queens','borough','llc','ulurp','repeal','action','cluster',
            'criterion','fall','month','annually','year','period','periodically','quarterly','biannually','biennially','biennial','biannually',
            'status','focus','offer','individual','pip','use','grant','lots','zrk','zrq','thereof','zmk','zmq','https','www','com','http',
            'today','tomorrow','yesterday','week','month','year','day','hour','minute','second','january','february','march','april','may',
            'june','july','august','september','october','november','december','monday','tuesday','wednesday','thursday','friday','saturday',
            'sunday','plaza','street','avenue','boulevard','road','place','court','square','lane','parkway','terrace','way','trail',
            'amp', 'community', 'thank', 'us', 'nyc', 'join', 'support', 'work', 'need', 'office', 'great', 'get', 'people', 'proud', 'time', 
            'nyccouncil', 'one', 'see', 'make', 'help', 'happy', 'many', 'thanks', 'must', 'first', 'yorkers', 'like'
            'communities','last',  'working', 'together', 'please', 'every', 'let', 'much',
            'good', 'know', 'love', 'family', 'back', 'colleagues', 'event', 'neighbors',
            'forward', 'joined', 'also', 'safe', 'members', 'celebrate',  'years', 'de', 'team', 'call', 'vote', 'continue', 'everyone', 
            'morning', 'right', 'budget', 'come', 'honor', 'would', 'member', 'next', 'keep', 'always',
            'hall','want','still','tonight','sure', 'got', 'pm','visit','another','like','ty','coney', 'current',
             'currently','agenda','nycmayor','hearing', 'bill','cmshahanahanif','ensure','bills','take','impact',
             'introduced', 'shahanafrombk', 'cmcrystalhudson',
            ]

In [10]:
tweet_df.shape

(145313, 10)

In [9]:
dsa_plus_tweets = tweet_df[tweet_df['rf_pca_predictions'].isin(dsa_plus)]

In [12]:
dsa_plus_tweets.head()

,Unnamed: 0,date,username,text,full_name,VotePersonName,faction,rf_predictions,knn_predictions,rf_pca_predictions
0,0,2023-03-14T23:23:01.000Z,CMCrystalHudson,Thank you Aging Chair @CMCrystalHudson for hol...,Crystal Hudson,Crystal Hudson,NaN,Affluent Liberals,Affluent Liberals,Alphabet Left
7,7,2023-03-14T22:09:14.000Z,CMPiSanchez,"Every resident, including those with disabilit...",Pierina Ana Sanchez,Pierina Ana Sanchez,NaN,Affluent Liberals,Affluent Liberals,Alphabet Left
10,10,2023-03-14T21:48:42.000Z,CMPiSanchez,The pernicious persistence of the wage gap bet...,Pierina Ana Sanchez,Pierina Ana Sanchez,NaN,Affluent Liberals,Affluent Liberals,Alphabet Left
14,14,2023-03-14T21:12:27.000Z,CabanD22,📣 RALLY ANNOUNCEMENT: NYC Council Members @CM...,Tiffany Cabán,Tiffany Cabán,DSA,DSA,DSA,DSA
27,27,2023-03-14T20:03:20.000Z,CMPiSanchez,"This weekend alongside @Kipsbayclub, I hosted ...",Pierina Ana Sanchez,Pierina Ana Sanchez,NaN,Affluent Liberals,Affluent Liberals,Alphabet Left


In [10]:
leg_words = ['bill','introduction','committee','act','sponsor','legislation','hearing','resolution']

In [11]:
# search text column for words in leg_words
dsa_plus_leg_tweets = dsa_plus_tweets[dsa_plus_tweets['text'].str.contains('|'.join(leg_words))]

In [12]:
# expand max column width to see full text
pd.set_option('display.max_colwidth', None)

In [27]:
dsa_plus_leg_tweets[['date','full_name','text']].sample(10)

,date,full_name,text
302,2023-03-09T16:56:51.000Z,Shahana K. Hanif,"Proud to write a letter to @SecretaryPete to support @NYC_DOT's efforts to fund an Urban Freight Mobility Collaborative. This initiative would develop actionable, innovative, equity-centered low and zero-emission freight delivery strategies. Read more 👇 https://t.co/3GM1WWyCeH"
81761,2021-12-24T00:04:30.000Z,Shahana K. Hanif,"Between Rapid NYC/Express Health’s now-removed citizenship question, CareCube’s surprise billing, &amp; LabQ’s false advertising, NY must hold private healthcare entities accountable &amp; make city-run #COVID19 testing sites more accessible so NYers do not have to rely on private sites."
138376,2021-10-01T23:57:48.000Z,Tiffany Cabán,And that’s the power of the DA in action. On any given night and on any given case they can simply decide not to send people to the death trap that is #Rikers.
68364,2021-04-21T15:57:50.000Z,Shahana K. Hanif,"If you want to take action before Thursday's rally or if you can't make it, email the general manager at iryna@thekestrel.com to demand they re-employ the workers and raise their salaries.\n\nWe must send a message to the garage owners that the community’s got the back of workers."
39039,2022-10-13T17:08:01.000Z,Alexa Avilés,"I also proudly support @CMRestler’s resolution in support of a strong Climate Action Plan for New York, and I support identifying underutilized City land for renewable energy generation, a bill introduced by @JustinBrannan.\n\nICYMI, my comments on the state's climate plan here: https://t.co/jz7Fh3lW2r"
111240,2021-03-11T19:38:01.000Z,Lincoln Restler,"@JabariBrisport You are the actual best! Thank you for being such a good friend, Jabari!"
26614,2022-02-23T16:27:57.000Z,Crystal Hudson,"Andrew is a data-driven comms &amp; research specialist. Prior to joining our team, he worked for voting rights groups &amp; municipal campaigns across the city. He is a musical theater enthusiast, soccer fan, avid reader &amp; pianist.\n\nFun Fact: “I’m a triplet (fraternal)!” 👶👧👶 https://t.co/DbXrhBFh5D"
66882,2021-04-15T21:33:46.000Z,Lincoln Restler,"Let’s not get it twisted. \n\nA $2200 per month studio is NOT affordable. \n\nIf I’m elected to the City Council, I will sponsor legislation to repeal tax breaks to developers for unaffordable apartments at 130% area median income. \n\nWe need truly affordable housing."
122384,2021-05-28T17:39:59.000Z,Alexa Avilés,"""Esto no es sobre mi como protagonista, sino que debemos hacerlo conjuntamente y la gente ve que mi interés es auténtico y que la gente más vulnerable es mi objetivo de trabajo, la liberación conjunta se dará cuando lo hagamos juntos.""\n\n📰 @ImpactoLatino: https://t.co/hV8bVIiWMI https://t.co/YongIonLkd"
142371,2021-09-23T18:21:59.000Z,Alexa Avilés,4/4 Councilmember @bradlander's bill would establish minimum per-trip payments for delivery workers.


In [13]:
def remove_punct(text):
    text_nopunct = ''.join([char for char in text if char not in string.punctuation])
    return text_nopunct

def remove_stopwords(text):
    words = [word for word in text.split() if word.lower() not in stopwords.words('english')]
    return words

def count_words(text):
    count_dict = Counter(text)
    return count_dict

In [14]:
def preprocess_data(documents):
    stop_words = set(stopwords.words('english'))
    stop_words.update(add_stop)
   

    tokenized_docs = [word_tokenize(doc.lower()) for doc in documents]
    processed_docs = [[word for word in doc if word not in stop_words and word.isalpha()] for doc in tokenized_docs]
    return processed_docs
def join_tokens(token_list):
    return ' '.join(token_list)

In [15]:
dsa_plus_leg_tweets['prepared_text'] = dsa_plus_leg_tweets['text'].apply(lambda x: preprocess_data([x])[0])

C:\Users\samtg\AppData\Local\Temp\ipykernel_26444\1720076862.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dsa_plus_leg_tweets['prepared_text'] = dsa_plus_leg_tweets['text'].apply(lambda x: preprocess_data([x])[0])


In [20]:
dsa_plus_tweets['prepared_text'] = dsa_plus_tweets['text'].apply(lambda x: preprocess_data([x])[0])

C:\Users\samtg\AppData\Local\Temp\ipykernel_26444\2913171403.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dsa_plus_tweets['prepared_text'] = dsa_plus_tweets['text'].apply(lambda x: preprocess_data([x])[0])


In [16]:
all_tokens = [token for sublist in dsa_plus_leg_tweets['prepared_text'] for token in sublist]
word_freq = Counter(all_tokens)

most_common_words = word_freq.most_common(20)  
common_words_list = [word for word, freq in most_common_words]
common_words_string = ', '.join(f"'{word}'" for word in common_words_list)
print(common_words_string)

'housing', 'workers', 'health', 'communities', 'care', 'access', 'fight', 'services', 'safety', 'justice', 'climate', 'families', 'immigrant', 'rights', 'across', 'fair', 'create', 'impacted', 'passed', 'provide'


In [21]:
all_tokens = [token for sublist in dsa_plus_tweets['prepared_text'] for token in sublist]
word_freq = Counter(all_tokens)

most_common_words = word_freq.most_common(20)  
common_words_list = [word for word, freq in most_common_words]
common_words_string = ', '.join(f"'{word}'" for word in common_words_list)
print(common_words_string)

'housing', 'workers', 'communities', 'park', 'fight', 'school', 'care', 'justice', 'health', 'services', 'safety', 'families', 'voteshekar', 'free', 'across', 'students', 'women', 'access', 'open', 'lincolnrestler'


In [17]:
housing = ['rent','landlord','housing','zoning','section 8','nycha']

In [19]:
streets = ['street','bike','bicycle','pedestrian']

In [24]:
dplt_streets = dsa_plus_leg_tweets[dsa_plus_leg_tweets['text'].str.contains('|'.join(streets))]

In [22]:
dp_streets = dsa_plus_tweets[dsa_plus_tweets['text'].str.contains('|'.join(streets))]

In [31]:
pattern = r'\b(?:' + '|'.join(re.escape(word) for word in housing) + r')\b'
dp_housing = dsa_plus_tweets[dsa_plus_tweets['text'].str.contains(pattern, regex=True)]

In [34]:
pattern = r'\b(?:' + '|'.join(re.escape(word) for word in streets) + r')\b'
dp_streets = dsa_plus_tweets[dsa_plus_tweets['text'].str.contains(pattern, regex=True)]

In [26]:
dp_housing = dsa_plus_tweets[dsa_plus_tweets['text'].str.contains('|'.join(housing))]
dplt_housing = dsa_plus_leg_tweets[dsa_plus_leg_tweets['text'].str.contains('|'.join(housing))]

In [35]:
all_tokens = [token for sublist in dp_streets['prepared_text'] for token in sublist]
word_freq = Counter(all_tokens)

most_common_words = word_freq.most_common(20)  
common_words_list = [word for word, freq in most_common_words]
common_words_string = ', '.join(f"'{word}'" for word in common_words_list)
print(common_words_string)

'bike', 'lanes', 'safety', 'protected', 'streets', 'pedestrian', 'trees', 'tree', 'ave', 'st', 'care', 'vendors', 'open', 'safer', 'park', 'space', 'ride', 'bridge', 'traffic', 'across'


In [37]:
dp_housing[['date','username','full_name','text']].sample(10)

,date,username,full_name,text
73482,2023-08-17T19:54:52.000Z,CMJulieWon,Julie Won,"who can help you better understand &amp; take next steps with your housing issue by providing legal information, resources &amp; quality referrals. Our housing lawyer will be in our office every Wednesday from 12:00-3:30 p.m. Contact our office to book an appointment! (2/2)"
35269,2022-12-08T15:58:06.000Z,CMShahanaHanif,Shahana K. Hanif,We support #FairChanceForHousing! This critical legislation affirms that discrimination has absolutely no place in our housing market. Thank you to @KeithPowersNYC and @NYCProgressives for leading the charge. ✊🏾🔑🏘️ https://t.co/83VKXcHZoz
23958,2022-03-28T23:01:46.000Z,CMCrystalHudson,Crystal Hudson,"Mark your 📅 for our Community Wellness Day this Thursday, 3/31 at First Baptist Church of Crown Heights! Stop by between 1pm-3pm for free food, clothes, housing support, preventative health screenings, and more. Tell a neighbor and bring a friend! https://t.co/UuPePnRxAz"
111945,2021-03-27T00:14:42.000Z,PiSanchezNYC,Pierina Ana Sanchez,We need support at all levels of government to stabilize our communities and reach housing dignity. \n\nJoin us this Saturday March 27th @ 10AM for a rally and press conference with leaders including \n@RepEspaillat and @RepRitchie!\n\nhttps://t.co/QkjfwpRQwx https://t.co/PhfGBBqmta
65912,2023-04-20T17:01:48.000Z,CabanD22,Tiffany Cabán,"Recipe for disaster. Want to intensify our homelessness, mental health, &amp; violence crises? Hike rents by double digits.\n\nThe purpose of our housing system should be housing people, not generating record profits for wealthy interests. #housingisahumanright https://t.co/xK8v8nZkOd"
112990,2021-03-12T00:15:53.000Z,ShahanaFromBK,Shahana K. Hanif,"We, as human beings, rely on animals for support, companionship, and to sustain our ecosystem.\n\nWe must proactively talk about animal rights as a part of our City's conversations around housing, worker’s rights, and climate change."
19699,2022-05-17T01:45:45.000Z,PiSanchezNYC,Pierina Ana Sanchez,"“Landlords, we need you to help stabilize our communities”: rent your vacant unit to a voucher holder and help an NYC family find a home! \n\nThank you @CMPiSanchez! #DoSomethingGoodForYourNeighborDay https://t.co/g01pDkAWVb"
97051,2021-07-08T20:14:33.000Z,CMAlexaAviles,Alexa Avilés,"3/3 The bill would prorate rent reductions whenever utility service fails. When you’re paying rent, you’re paying for the basic necessities all human beings deserve. An apartment without reliable heat, hot water, gas or water isn’t a proper home. You deserve better."
42889,2022-08-16T20:39:52.000Z,alexaforcouncil,Alexa Avilés,"I stood with tenants of 702 54 St early this morning to rally before their day in court.\n\nFires expose so many systemic wrongs with housing in this city. Despite all the challenges, tenants are united and demanding a court-ordered timeline for repairs. ✊ https://t.co/zCuVqnH4n0 https://t.co/07CD8vTHkT"
58771,2022-01-17T18:04:39.000Z,LincolnRestler,Lincoln Restler,"This #MLKDay, we’re out across the district connecting tenants in our neighborhood with critical resources to protect them from evictions. It’s our hope to honor the legacy of Dr. Martin Luther King Jr., because housing justice is racial justice. https://t.co/wvZKJfLtSb"


In [61]:
# get all tweets from tweet_df from August 11, 2022
tweet_df['date'] = pd.to_datetime(tweet_df['date'])
tweet_df['date'].dtype

datetime64[ns, UTC]

In [67]:
tweet_df[['date','full_name']].head()

,date,full_name
0,2023-03-14 23:23:01+00:00,Crystal Hudson
1,2023-03-14 22:56:52+00:00,Eric Dinowitz
2,2023-03-14 22:50:55+00:00,Adrienne E. Adams
3,2023-03-14 22:35:06+00:00,Marjorie Velázquez
4,2023-03-14 22:34:36+00:00,Marjorie Velázquez


In [68]:
# get all tweets from tweet_df from August 11, 2022. Date is in pd.datetime format
august = tweet_df[tweet_df['date'].dt.strftime('%m-%d') == '08-11']

In [81]:
bill_words = ['nonprofit','non-profit','land','developer','equity']

In [82]:
aug_housing = august[august['text'].str.contains('|'.join(bill_words))]

In [79]:
aug_housing = august[august['text'].str.contains('intro 637')]

In [84]:
aug_housing.head(1)

,Unnamed: 0,date,username,text,full_name,VotePersonName,faction,rf_predictions,knn_predictions,rf_pca_predictions
13543,13543,2022-08-11 20:40:03+00:00,LincolnRestler,"With @CMRestler and 17 of my Council colleagues, we are introducing intro 637 to ensure public land is used for public good. Instead of giving away public land to for-profit developers, this bill would ensure it goes to nonprofits, community land trusts, and limited equity coops. https://t.co/iEhYLKyDoh",Lincoln Restler,Lincoln Restler,NaN,Affluent Liberals,Affluent Liberals,Alphabet Left


Search for councilwoman Aviles's social housing bill

In [89]:
int0932 = tweet_df[tweet_df['date'].dt.strftime('%m-%d') == '02-16']

In [91]:
int0932_house = int0932[int0932['text'].str.contains('housing')]

In [92]:
int0932_house

,Unnamed: 0,date,username,text,full_name,VotePersonName,faction,rf_predictions,knn_predictions,rf_pca_predictions
1883,1883,2023-02-16 18:52:02+00:00,CMSalamancaJr,"Being Chair of the Committee on Land Use, I enjoy speaking with &amp; visiting my colleagues to discuss zoning regulations in their respective districts. This week, I was in #District44 w/@KalmanYeger to explore the community’s needs surrounding affordable housing. https://t.co/sOGBrh95Ga","Rafael Salamanca, Jr.","Rafael Salamanca, Jr.",NaN,Affluent Liberals,Affluent Liberals,Affluent Liberals
1910,1910,2023-02-16 16:18:37+00:00,CMSandyNurse,The rent is too damn high and keeping and affording your housing is the #1 issue in NYC! Thank you @CMSandyNurse! @GovKathyHochul Pass &amp; sign #GoodCause! https://t.co/uAmfITRFyj,Sandy Nurse,Sandy Nurse,Hybrid Progressives,Hybrid Progressives,Hybrid Progressives,Hybrid Progressives
1913,1913,2023-02-16 15:55:59+00:00,CMSandyNurse,But good cause is just the first step -- the FLOOR. We need to transform our housing system to put people's homes over real estate. #OurHomesOurPower\n\n@SandyforCouncil's op-ed today on #GoodCause and the path to social housing is a must read: \n\nhttps://t.co/y9VYUUYI0U,Sandy Nurse,Sandy Nurse,Hybrid Progressives,Hybrid Progressives,Hybrid Progressives,Hybrid Progressives
1915,1915,2023-02-16 15:40:43+00:00,nycpa,"Deputy Public Advocates reporting for duty.\n\nRepresenting our Community Engagement team, the three Deputy Public Advocates here work with New Yorkers on education, employment opportunity and labor rights, housing and environmental justice, and infrastructure. \n\n#Squad ☮️ https://t.co/PXReOsMR7y",Jumanaae Williams,NaN,NaN,NaN,NaN,NaN
1918,1918,2023-02-16 15:09:08+00:00,SandyforCouncil,"Shout out to my State Senator @JuliaCarmel__ who carries the bill in the Senate! And to all the organizers and tenants hiking up to Albany for years to demand basic protections and dignity for safe, secure and affordable housing. We deserve better! https://t.co/ysyFoK47jR",Sandy Nurse,Sandy Nurse,Hybrid Progressives,Hybrid Progressives,Hybrid Progressives,Hybrid Progressives
1923,1923,2023-02-16 14:09:54+00:00,alexaforcouncil,The rent is too damn high and keeping and affording your housing is the #1 issue in NYC! Thank you @CMSandyNurse! @GovKathyHochul Pass &amp; sign #GoodCause! https://t.co/uAmfITRFyj,Alexa Avilés,Alexa Avilés,DSA,DSA,DSA,DSA
26910,26910,2022-02-16 19:12:32+00:00,NYCCouncil38,"We toured three developments yesterday: Redfern, Ocean Bay, and Carleton Manor.\n\nResidents at Carleton Manor recently went months without heat. In an advanced country like ours, this should never happen. Not in private housing, certainly not in public housing. https://t.co/tNTpg56pSf",Alexa Avilés,Alexa Avilés,DSA,DSA,DSA,DSA
26914,26914,2022-02-16 19:08:18+00:00,alexaforcouncil,"We toured three developments yesterday: Redfern, Ocean Bay, and Carleton Manor.\n\nResidents at Carleton Manor recently went months without heat. In an advanced country like ours, this should never happen. Not in private housing, certainly not in public housing. https://t.co/tNTpg56pSf",Alexa Avilés,Alexa Avilés,DSA,DSA,DSA,DSA
26919,26919,2022-02-16 18:24:20+00:00,CMPiSanchez,"Did you know there are more than 3,000 affordable housing opportunities on Housing Connect right now?! Need help getting started? This video explains what affordable housing is, how to create an account on Housing Connect and what happens after you apply: https://t.co/hmLc0QBogS https://t.co/SkEkYGTO1X",Pierina Ana Sanchez,Pierina Ana Sanchez,NaN,Affluent Liberals,Affluent Liberals,Alphabet Left
26929,26929,2022-02-16 17:08:05+00:00,CouncilwomanKrj,"It was my honor to speak in support of project Castle III at today’s Subcommittee on Landmarks, Public Sittings, and Dispositions. This is a step towards #actuallyaffordablehousing and a 🌳#greener and cleaner city. 🧵 https://t.co/40qq8r8rZH",Kristin Richardson Jordan,Kristin Richardson Jordan,NaN,Hybrid Progressive